<a href="https://colab.research.google.com/github/Mayarmohamed/Graduation-Project-Prediction-of-drug-Sensitivity-in-AML-patients/blob/main/Prediction_of_Drug_sensitivity_for_AML_Patients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**To seach for the optimum paramaters that increase the r2 score**

In [ ]:
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from  openpyxl import load_workbook
import numpy as np
from pandas import DataFrame
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.feature_selection import mutual_info_regression
import matplotlib.pyplot as plt
import os 
import math

directory='/content/drive/MyDrive/Colab Notebooks/Model Files'
drugs=pd.ExcelFile(directory+'/BEAT_AML_DrugResponse_Kinase.xlsx')
df = pd.read_excel(drugs, 'All Drugs')
df1=df[df['inhibitor']=='Pelitinib (EKB-569)']
expression=pd.read_excel(directory+'/Coding_Genes.xlsx').T

#Data Preprocessing
new_header = expression.iloc[0] #grab the first row for the header
expression = expression[1:] #take the data less the header row
expression.columns = new_header #set the header row as the df header
data=expression.loc[expression.index.intersection(df1['lab_id']),:]
df1 = df1.set_index('lab_id')
dru=df1.loc[df1.index.intersection(data.index)]
print(dru)

#Mutual Information
mi = mutual_info_regression(data, dru['auc'],random_state=42)
mi = pd.DataFrame(mi)
mi.insert(0, "Genes", X_train.columns, True)
mi.columns=["Genes","data"]
z1=mi.sort_values(by="data",ascending=False)
z=z1[z1["data"]>=0.1]

#Data Splitting
X_train, X_valid, y_train, y_valid = train_test_split(data, dru['auc'], test_size=0.3, random_state=0)
r2={}

#Mutual Information Data 
for mutual in range(100,350):
  index_=z1.iloc[[i for i in range(mutual)]].index.values
  x_train=X_train.iloc[:,index_]
  x_test=X_valid.iloc[:,index_]
  #Random forest Model
  for trees in range(10,150):
    rf = RandomForestRegressor(random_state=0,max_features= 'sqrt',n_estimators=trees)
    rf.fit(x_train, y_train)
    reg_prediction=rf.predict(x_test)
    r2s= r2_score(y_valid, reg_prediction)
    r2[str(trees)+' & '+str(mutual)]=r2s
    print(r2s,'\t',mean_squared_error(y_valid,reg_prediction))

#Plotting the r2 score for tested RF paramaters
plt.plot(list(r2.keys()),list(r2.values()))
plt.show()
Keymax = max(r2, key=r2.get)
print(Keymax)
print(r2.get(Keymax))



**Evaluation of Each model and extract the feature importance**

In [ ]:
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from  openpyxl import load_workbook
import numpy as np
from pandas import DataFrame
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.feature_selection import mutual_info_regression
import matplotlib.pyplot as plt
import os 
import math

directory='/content/drive/MyDrive/Colab Notebooks/Model Files'
drugs=pd.ExcelFile(directory+'/BEAT_AML_DrugResponse_Kinase.xlsx')
df = pd.read_excel(drugs, 'All Drugs')
df1=df[df['inhibitor']=='Afatinib (BIBW-2992)'] #or Gefitinib Erlotinib Lapatinib Pelitinib (EKB-569)
expression=pd.read_excel(directory+'/Coding_Genes.xlsx').T


#Data Preprocessing
new_header = expression.iloc[0] #grab the first row for the header
expression = expression[1:] #take the data less the header row
expression.columns = new_header #set the header row as the df header
data=expression.loc[expression.index.intersection(df1['lab_id']),:]
df1 = df1.set_index('lab_id')
dru=df1.loc[df1.index.intersection(data.index)]
print(dru)

#Data Splitting

X_train, X_valid, y_train, y_valid = train_test_split(data, dru['auc'], test_size=0.3, random_state=0)

#Mutual Information
mi = mutual_info_regression(data, dru['auc'],random_state=42)
mi = pd.DataFrame(mi)
mi.insert(0, "Genes", X_train.columns, True)
mi.columns=["Genes","data"]
z1=mi.sort_values(by="data",ascending=False)
z=z1[z1["data"]>=0.1]


#Mutual Information Data 
index_=z1.iloc[[i for i in range(156)]].index.values
x_train=X_train.iloc[:,index_]
x_test=X_valid.iloc[:,index_]
#Random forest Model
rf = RandomForestRegressor(random_state=0,max_features= 'auto',
                           n_estimators=7)
rf.fit(x_train, y_train)
reg_prediction=rf.predict(x_test)
r2s= r2_score(y_valid, reg_prediction)
mse=mean_squared_error(y_valid,reg_prediction)
print("The r2 score is ",r2s,'\nThe MSE is',mse,'\nThe RMSE is',sqrt(mse))


sorted_idx = rf.feature_importances_.argsort()
feature_list=expression.columns
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature,importance) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances[:5]];

